## 1. Montar Google Drive y cargar modelo + scaler

In [1]:
from google.colab import drive
import joblib

# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Ruta a la carpeta donde guardaste modelo y scaler
model_path = "/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_practica1/modelo"

# Cargar modelo Random Forest
rf_model = joblib.load(f"{model_path}/modelo_rf.pkl")

# Cargar scaler
scaler = joblib.load(f"{model_path}/scaler.pkl")

print("Modelo y scaler cargados correctamente.")

Modelo y scaler cargados correctamente.


## 2. Instalar Flask y flask-ngrok (solo una vez)

In [3]:
!pip install flask
!pip install pyngrok

## Autenticación de ngrok

Ir al panel de ngrok: https://dashboard.ngrok.com/get-started/your-authtoken

In [4]:
from pyngrok import conf
conf.get_default().auth_token = "2ws6ObilHgxHH5kyOdXf4Zq13Z2_47dq7qtXoEnharEskeBAE"

## Cerrar todos los túneles activos

In [5]:
from pyngrok import ngrok

# Cerrar todos los túneles activos
ngrok.kill()  # Cierra el agente y libera los túneles

##  3. Importar y configurar Flask + pyngrok

In [6]:
from flask import Flask, request, render_template
from pyngrok import ngrok
import threading
import numpy as np
from flask import url_for

# Inicializar Flask
app = Flask(
    __name__,
    template_folder="/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_practica1/templates",
    static_folder="/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/MIAA_practica1/static"
)


@app.route("/")
def formulario():
    return render_template("formulario.html")

@app.route("/predict")
def predict():
    try:
        # Captura de parámetros
        ph = float(request.args.get("ph"))
        hardness = float(request.args.get("hardness"))
        solids = float(request.args.get("solids"))
        chloramines = float(request.args.get("chloramines"))
        sulfate = float(request.args.get("sulfate"))
        conductivity = float(request.args.get("conductivity"))
        organic_carbon = float(request.args.get("organic_carbon"))
        trihalomethanes = float(request.args.get("trihalomethanes"))
        turbidity = float(request.args.get("turbidity"))

        # Índice de sales (calculado automáticamente)
        indice_sales = solids / conductivity if conductivity != 0 else 0

        # Crear vector de entrada
        inputs = [
            ph, hardness, solids, chloramines, sulfate,
            conductivity, organic_carbon, trihalomethanes, turbidity,
            indice_sales
        ]

        entrada = scaler.transform([inputs])
        pred = rf_model.predict(entrada)[0]

        if pred == 1:
            resultado_texto = "Con los valores de los parámetros ingresados el agua es Potable ✅"
            imagen_url = url_for('static', filename='images/potable.jpg')
        else:
            resultado_texto = "Con los valores de los parámetros ingresados el agua es No Potable ❌"
            imagen_url = url_for('static', filename='images/no_potable.jpg')

        return render_template("resultado.html", resultado_texto=resultado_texto, imagen_url=imagen_url)

    except Exception as e:
        return f"<p>Error: {e}</p>"

# Ejecutar Flask con ngrok
def run():
    app.run()

public_url = ngrok.connect(5000)
print("Tu app está disponible en:", public_url)

thread = threading.Thread(target=run)
thread.start()

Tu app está disponible en: NgrokTunnel: "https://b6e3-35-234-32-37.ngrok-free.app" -> "http://localhost:5000"
